In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 512

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [1]:
class CreateCounter:
    count = 0  # This is a class attribute.
    def __init__(self):
        CreateCounter.count += 1

print('Objects created:', CreateCounter.count)
a = CreateCounter()
b = CreateCounter()
c = CreateCounter()
print('Objects created:', CreateCounter.count)

Objects created: 0
Objects created: 3


In [4]:
# functions -> oop -> modularity, code reuse
class Net(nn.Module):  # class: an empty form
    def __init__(self):  # “double underscore” to “dunder” methods; self refers to an object
        super().__init__()  # to reach parent class's methods and attributes
        self._conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=0)  # private attribute
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 64, 5)
        self.fc1 = nn.Linear(64 * 5 * 5, 130)
        self.do1 = nn.Dropout(p=0.4)
        self.fc2 = nn.Linear(130, 100)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x):  # method
        x = self.pool(F.relu(self._conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.do1(self.fc1(x)))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    # Instance method
    def printPool1(self):
        print("instance method", self)

    @staticmethod      # can be called w/o an object; If you do decide to use one, you should strongly consider just creating a regular function instead.
    def printPool2():  # From other langs.
        print("static method")


#net = Net()  # object: a filled form
#net = net.to("cuda")  # to method from nn.Module

#net._conv1 = 123 -> private variable (or method), should be used only for internal use.

In [7]:
Net().printPool2()

static method


In [ ]:
#net = Net()
net.printPool1()

In [ ]:
# Encapsulation: box up, information hiding

In [9]:
# Polymorphism: can handle objects of many different types
len("abcdefg")

7

In [11]:
len([1,2,3,4,5,6,'7'])

7

In [12]:
# Operator overloading
3 + 4

7

In [13]:
'3' + '4'

'34'

In [2]:
# Inheritance:

class ParentClass:
  def parentFunc(self):
    print('I am parent.')

class ChildClass(ParentClass):
  def childFunc(self):  #
    print("I am child.")

class GrandchildClass(ChildClass):
  def grandChildFunc(self):
    print('I am grandchild.')
  def childFunc(self):  # Method overriding; polymorphism
    print("Overridden childFunc.")

In [ ]:
parent = ParentClass()
parent.parentFunc()

In [ ]:
child = ChildClass()
child.parentFunc()
child.childFunc()

In [ ]:
grandchild = GrandchildClass()
grandchild.parentFunc()
grandchild.childFunc()
grandchild.grandChildFunc()

In [6]:
parent.childFunc()

AttributeError: 'ParentClass' object has no attribute 'childFunc'

In [ ]:
array = np.array([1,2,3,4,5])

In [ ]:
array.sum()

In [ ]:
np.sum(array)

In [35]:
class ClassProperties:
    def __init__(self):
        self.attribute = 10
    
    @property
    def attribute(self):  # Getter method.
        return self._attribute
        
    @attribute.setter
    def attribute(self, value):  # Setter method.
        if not isinstance(value, int):
            raise TypeError("attribute must be an integer")
        if value < 0:
            raise ValueError("attribute must be positive")
        self._attribute = value
    
    @attribute.deleter
    def attribute(self):  # Deleter method.
        del self._attribute

    def __str__(self):
        return f"attribute: {self.attribute}"
    
    def __add__(self, other):  # Operator overloading
        a = ClassProperties()
        a.attribute = self.attribute * other.attribute
        return a
    
    def __mul__(self, other):  # Operator overloading
        a = ClassProperties()
        a.attribute = self.attribute + other.attribute
        return a

obj = ClassProperties()
print(obj.attribute)  # calls the getter method 
obj.attribute = 11  # calls the setter method
print(obj.attribute)
#del obj.attribute  # calls the deleter method

10
11


In [36]:
obj2 = ClassProperties()
obj2.attribute = 20

sum = obj + obj2
mul = obj * obj2
print(f"'Summation': {sum}\n'Multiplication': {mul}")

'Summation': attribute: 220
'Multiplication': attribute: 31


In [ ]:
from torch import optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [ ]:
net.train()

for epoch in range(10):

    loss_epoch = 0.0
    for i, data in enumerate(trainloader):

        inputs, labels = data[0].to("cuda"), data[1].to("cuda")

        # zero the parameter gradients
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        loss_epoch += loss.item()

    print(f'Epoch {epoch + 1} - Loss: {loss_epoch / (len(trainloader)):.3f}')

In [ ]:
net.eval()

correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to("cuda"), data[1].to("cuda")
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')